In [1]:
import numpy as np
import cv2
from keras.layers import Conv2D, Input, Layer
import keras.layers
from keras.models import Model
import tensorflow as tf

/home/chandu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Custom layer for bicubic interpolation 

class InterpolateTensor(Layer):
    
    # x, y : final size
    # c : channels
    def __init__(self, x, y, c, **kwargs):
        self.x = x
        self.y = y
        self.channels = c
        self.trainable = False
        super(InterpolateTensor, self).__init__(**kwargs)
    
    def call(self, x):
        return tf.image.resize_bicubic(x, (self.x, self.y))
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.x, self.y, self.channels)
    

In [3]:
class RecCNN:
    
    def __init__(self, c):
        self.channels = c
        
    def sisr(self, first):
        
        # similar to srcnn model 
        interp = InterpolateTensor(375, 500, 3)(first)
        dup = interp
        conv1 = Conv2D(filters=64, kernel_size=(5,5), activation='relu', padding='same')(interp)
        conv2 = Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same')(conv1)
        res = Conv2D(self.channels, kernel_size=(3,3), activation='sigmoid', padding = 'same')(conv2)
        last = keras.layers.add([dup, res])
        
        return last
    
        

In [4]:
# inp2 = Input(shape=(25,50,3))
# rec_cnn = RecCNN(3)
# model_reccnn = Model(inp2, rec_cnn.sisr(inp2))
# model_reccnn.compile(optimizer='adam', loss='mean_squared_error')
# model_reccnn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 25, 50, 3)    0                                            
__________________________________________________________________________________________________
interpolate_tensor_1 (Interpola (None, 375, 500, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 375, 500, 64) 4864        interpolate_tensor_1[0][0]       
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 375, 500, 32) 18464       conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (